In [ ]:
!pip install -q datasets pandas pymongo sentence_transformers

In [ ]:
!pip install -q accelerate

In [ ]:
!pip install pymongo[rsv]

In [ ]:
import pymongo
import pandas as pd


In [ ]:
from google.colab import userdata

In [ ]:
client = pymongo.MongoClient(userdata.get('mongo_url'))

SecretNotFoundError: Secret mongo_url does not exist.

In [ ]:
db = client["hoanghamonbilenew"]
collection = db['embedding_for_vector_search']

In [ ]:
def to_markdown(text):
  text = text.replace('.',' *')
  return Markdown(textwrap.indent(text,'>', predicate = lambda _: True))

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("keepitreal/vietnamese-sbert")

In [ ]:
def get_embedding(text):
  if not text.strip():
    print("Attempted to get embedding for empty text.")
    return []
    embedding = embedding_model.encode(text)
    return embedding.tolist()

In [ ]:
#Vector Search function here
def vector_search(user_query, collection, limit = 4):
  #Generate embedding for user query
  query_embedding = get_embedding(user_query)
  if query_embedding is None:
    return "Invalid query or embedding generation failed."
  #Define the vector search pipeline
  vector_search_stage = {
      "$vectorSearch":{
          "index": "vector_index",
          "queryVector": query_embedding, #Vector embedding của câu truy vấn.
          "path": "embedding", # Đây là tên trường trong document chứa embedding
          "numCandidates": 400, #ọc trước 400 documents gần nhất (approximate)
          "limit":limit,
      }
  }

  unset_stage = {
      "$unset":"embedding"
  }
  project_stage = {
      "$project":{
          "_id": 0, # không trả về trường ID
          "title": 1, # 0:không trả về, 1=có trả về
          "product_specs":1,
          "color_options": 1,
          "current_price": 1,
          "product_promotion":1,
          "score":{"$meta":"vectorSearchScore"} #Tính điểm cosine similariry sử dụng cách tính vectorSearchScore của
      }
      }
  pipeline = [vector_search_stage, unset_stage, project_stage]
  #Execute the search
  results = collection.aggregate(pipeline)
  return list(results)

In [ ]:
def get_search_result(query, collection):
  get_knowledge = vector_search(query, collection, 10)
  search_result = ""
  i=0
  for result in get_knowledge:
    print(result)
    if result.get('current_price'):
      i+=1
      search_result += f"\n {i}) Ten: {result.get('title')}"

      if result.get('current_price'):
        search_result += f"\n Gia: {result.get('current_price')}"
      else:
        #Mock up data
        #Retrieval model pricing from the internet
        search_result += f"\n Gia: Liên lạc tới đường dây nóng của BHXH địa phương để trao đổi thêm"
      if result.get('product_promotion'):
        search_result += f", Ưu đãi: {result.get('product_promotion')}"
      return search_result

In [ ]:
query = "Nhận chế độ bảo hiểm một lần"
source_information = get_search_result(query.lower(),collection).replace('<br>', '\n')
source_information

NameError: name 'collection' is not defined

In [ ]:
combined_information = f"abc" #Prompt from Tung here
print('==== LLMs Inference')
print(combined_information)

==== LLMs Inference
abc


In [ ]:
!pip install pathLib

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace("."," *")
  return Markdown(textwrap.indent(text, ">", predicate = lambda _: True))

In [ ]:
# genai.configure(api_key=userdata.get('GEMINI_API_KEY')) #get api key from Gooogle Studio
import google.generativeai as genai

genai.configure(api_key="GEMINI_API_KEY") #Update new Google Studio API key here

model = genai.GenerativeModel(model_name="gemini-2.0-flash")


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
%%time
reponse = model.generate_content(combined_information)
to_markdown(reponse.text)



BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.

In [ ]:

# #If we want to use another model from huggingface
# from huggingface_hub import login
# # Login using API key
# api_token = userdata.get('HUGGINGFACE_API_KEY')
# login(token=api_token)

# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# #GPU Enabled use below
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")


In [ ]:
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda") #run by GPU
reponse = model.generate(**input_ids, max_new_tokens = 100)
print(tokenizer.decode(reponse[0]))


NameError: name 'tokenizer' is not defined